In [1]:
import pandas as pd
import numpy as np

In [13]:
with open('cl.projbands', 'r') as f:
    lines = f.readlines()

# store the comment lines
comments = []
for line in lines:
    if line.startswith('#'):
        comments.append(line)
    else:
        break

# other lines store the data
data = []
for line in lines:
    if not line.startswith('#'):
        data.append(line)


In [15]:
# convert the list of strings to a list of lists
data = [line.split() for line in data]

In [16]:
# conver the 2d list to a dataframe, and convert the strings to floats
df = pd.DataFrame(data).astype(float)

In [34]:
df


,0,1,2,3,4,5,6,7,8,9,...,770,771,772,773,774,775,776,777,778,779
0,1.0,0.0000,-19.26411,0.972,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0000,-19.26411,0.972,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0000,-19.26401,0.974,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0000,-19.26401,0.974,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0000,-19.26272,0.974,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24595,41.0,5.5936,4.82218,0.770,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
24596,41.0,5.5936,5.00484,0.752,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
24597,41.0,5.5936,5.00484,0.752,0.0,0.0,0.0,0.000,0.000,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
24598,41.0,5.5936,5.00564,0.755,0.0,0.0,0.0,0.002,0.002,0.0,...,0.0,0.0,0.002,0.002,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
max(df[3])

1.0

In [63]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt

def plot_for_k(k_select):
    temp_contrib_c_dict = {}
    temp_contrib_v_dict = {}
    
    f = h5.File('eigenvectors.h5','r')
    evals = f['exciton_data/eigenvalues'][()]
    evc = f['exciton_data/eigenvectors'][()]
    evc = evc[0,:,:,:,:,0,:]
    (nS, nk, nc, nv, _) = evc.shape

    if k_select == -1:
        pass
    else:
        evc = evc[:,[k_select],:,:,:]

    for iN_S in range(nS):
        if e_min<evals[iN_S]<e_high:
            temp_contrib_cv = np.sum(abs(evc[iN_S,:,:,:,0]+evc[iN_S,:,:,:,1]*1j)**2, axis=0) # Summarize_eigenvectors and Ploteigenvectors - For a certain k and c, calculating the sum of Acvk across all vs, and vice versa, this is the correct way to do it, because for a certain k, valence band pdos idoes not change when we fix a v, and vice versa, But you ignore the correlation between c and v, thus you miss the ECF, but you can only plot ECF_noshift
            temp_contrib_v = np.sum(temp_contrib_cv, axis=0)
            temp_contrib_c = np.sum(temp_contrib_cv, axis=1)

            temp_contrib_c_dict[(iN_S,k_select)] = temp_contrib_c
            temp_contrib_v_dict[(iN_S,k_select)] = temp_contrib_v
    
        
    return temp_contrib_c_dict, temp_contrib_v_dict

In [64]:
scale = 10E2
e_min = 0
e_high = 5

for k in range(0,1):
    temp_contrib_c_dict = {}
    temp_contrib_v_dict = {}
    temp_contrib_c_dict, temp_contrib_v_dict = plot_for_k(k)
    #print(temp_contrib_c_dict)
    #print(temp_contrib_v_dict)

In [65]:
temp_contrib_c_dict

{(0,
  0): array([1.63549265e-03, 1.53034567e-03, 2.38308886e-03, 2.37721058e-03,
        2.64567592e-03, 2.64368169e-03, 1.62896102e-03, 1.62410469e-03,
        6.15226090e-04, 6.16350388e-04, 5.26247187e-04, 5.26013880e-04,
        1.73794237e-04, 1.74056345e-04, 1.12543353e-04, 1.12445585e-04,
        1.43653632e-05, 1.43850657e-05, 2.31587293e-05, 2.31303476e-05,
        3.65749825e-06, 3.62315701e-06, 6.72281291e-06, 6.73260772e-06,
        6.14821997e-06, 6.14732956e-06, 1.06257913e-05, 1.06142721e-05,
        3.79200775e-06, 3.79794429e-06, 3.18105575e-06, 3.18209876e-06,
        1.18586976e-05, 1.18791144e-05, 2.49392224e-06, 2.48822676e-06,
        8.06140269e-06, 8.05740656e-06, 1.95643245e-06, 1.94802626e-06]),
 (1,
  0): array([1.67523066e-03, 1.70964156e-03, 2.75338164e-03, 2.75399360e-03,
        2.49932535e-03, 2.49787734e-03, 1.66727108e-03, 1.66104016e-03,
        5.88940198e-04, 5.89771919e-04, 5.54367394e-04, 5.54338016e-04,
        1.68293149e-04, 1.68551304e-04, 1.

In [43]:
# find the corresponding pdos for each k from df
# Count the number of k points
n_k = len(df[0].unique())
n_k

41

In [46]:
# For every k, find the corresponding pdos, before that, we need to isolating only the middle 80 rows for each k, corresponding to the 40 valence and 40 conduction bands
# 首先把 df 根据第 0 列进行分组，然后对每一组进行操作
# 然后对每一组，取出中间 80 行, 也即 fermi - 40 : fermi + 40
# ! 定位出 fermi 的位置, 也即根据 df[2] 从负变成正的那个位置, 这里先 hard-coded 算了

fermi = 544 # VBM - start + 1 
middle_df = df.groupby(0).apply(lambda x: x.iloc[fermi-40:fermi+40,:])
middle_df

0       1        2      3    4    5      6      7      8    \
0                                                                         
1.0  504     1.0  0.0000 -1.11855  0.910  0.0  0.0  0.002  0.002  0.002   
     505     1.0  0.0000 -1.11855  0.910  0.0  0.0  0.002  0.002  0.002   
     506     1.0  0.0000 -1.11832  0.916  0.0  0.0  0.002  0.002  0.002   
     507     1.0  0.0000 -1.11832  0.916  0.0  0.0  0.002  0.002  0.002   
     508     1.0  0.0000 -1.08475  0.908  0.0  0.0  0.003  0.005  0.005   
...          ...     ...      ...    ...  ...  ...    ...    ...    ...   
41.0 24579  41.0  5.5936  4.18853  0.792  0.0  0.0  0.000  0.000  0.000   
     24580  41.0  5.5936  4.37509  0.838  0.0  0.0  0.000  0.000  0.000   
     24581  41.0  5.5936  4.37509  0.838  0.0  0.0  0.000  0.000  0.000   
     24582  41.0  5.5936  4.37588  0.832  0.0  0.0  0.000  0.001  0.001   
     24583  41.0  5.5936  4.37588  0.832  0.0  0.0  0.000  0.001  0.001   

              9    ...  770  771  772  773  774  775  776  777  778  779  
0                  ...                                                    
1.0  504    0.002  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     505    0.002  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     506    0.002  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     507    0.002  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     508    0.003  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
41.0 24579  0.000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     24580  0.000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     24581  0.000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     24582  0.000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
     24583  0.000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3280 rows x 780 columns]

In [52]:
middle_df.loc[1]
middle_df.loc[1][2].values

array([-1.11855, -1.11855, -1.11832, -1.11832, -1.08475, -1.08475,
       -1.08465, -1.08465, -1.0001 , -1.0001 , -1.00001, -1.00001,
       -0.9879 , -0.9879 , -0.98769, -0.98769, -0.6185 , -0.6185 ,
       -0.61833, -0.61833, -0.59292, -0.59292, -0.59277, -0.59277,
       -0.47366, -0.47366, -0.47359, -0.47359, -0.41901, -0.41901,
       -0.41897, -0.41897, -0.24652, -0.24652, -0.24647, -0.24647,
       -0.21691, -0.21691, -0.21685, -0.21685,  2.68539,  2.68539,
        2.68597,  2.68597,  2.91862,  2.91862,  2.9191 ,  2.9191 ,
        2.98328,  2.98328,  2.98362,  2.98362,  3.16894,  3.16894,
        3.16919,  3.16919,  3.45468,  3.45468,  3.45562,  3.45562,
        3.49358,  3.49358,  3.49488,  3.49488,  4.08349,  4.08349,
        4.08369,  4.08369,  4.1677 ,  4.1677 ,  4.16796,  4.16796,
        4.3169 ,  4.3169 ,  4.31745,  4.31745,  4.42533,  4.42533,
        4.42584,  4.42584])

In [57]:
middle_df.loc[1]

,0,1,2,3,4,5,6,7,8,9,...,770,771,772,773,774,775,776,777,778,779
504,1.0,0.0,-1.11855,0.910,0.0,0.0,0.002,0.002,0.002,0.002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505,1.0,0.0,-1.11855,0.910,0.0,0.0,0.002,0.002,0.002,0.002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
506,1.0,0.0,-1.11832,0.916,0.0,0.0,0.002,0.002,0.002,0.002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,1.0,0.0,-1.11832,0.916,0.0,0.0,0.002,0.002,0.002,0.002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,1.0,0.0,-1.08475,0.908,0.0,0.0,0.003,0.005,0.005,0.003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,1.0,0.0,4.31745,0.730,0.0,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,1.0,0.0,4.42533,0.771,0.0,0.0,0.000,0.001,0.001,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,1.0,0.0,4.42533,0.771,0.0,0.0,0.000,0.001,0.001,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
582,1.0,0.0,4.42584,0.768,0.0,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
temp_contrib_c_dict[(0,1)]

{(0,
  1): array([4.32620880e-04, 4.29688394e-04, 5.15783590e-04, 5.17895780e-04,
        1.34035181e-03, 1.33851151e-03, 9.33807221e-04, 9.30102402e-04,
        4.63281330e-04, 4.60631515e-04, 3.75478499e-04, 3.75300614e-04,
        1.50692758e-04, 1.47823037e-04, 1.00173223e-04, 1.00558576e-04,
        1.95338495e-05, 1.93315164e-05, 2.68065242e-05, 2.65044786e-05,
        3.14533829e-06, 3.13530280e-06, 5.01266439e-06, 4.94901531e-06,
        3.41235190e-06, 3.59367330e-06, 1.19012358e-05, 1.17212935e-05,
        4.97269050e-06, 4.99371620e-06, 5.01267016e-06, 4.93202491e-06,
        5.19443938e-06, 5.29171996e-06, 2.78117892e-06, 2.76687186e-06,
        4.52321235e-06, 4.53991852e-06, 1.79706940e-06, 1.78469321e-06]),
 (1,
  1): array([4.31136385e-04, 4.34637621e-04, 5.48717492e-04, 5.45016847e-04,
        1.27718505e-03, 1.27543459e-03, 9.52038069e-04, 9.46944423e-04,
        4.56086161e-04, 4.55336977e-04, 3.88207611e-04, 3.89134875e-04,
        1.45319978e-04, 1.46910544e-04, 1.